In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# Read the csv file
df = pd.read_csv('data.csv', header=None)

# Remove empty rows
df = df.replace(np.nan,0)

# Convert the dataframe to a numpy array
data = df.to_numpy()

In [2]:
data

array([[0.0, 'R', 'P', ..., 'R', 'R', 'R']], dtype=object)

In [3]:
data.shape

(1, 200000)

In [4]:
Mapping = {'R':1,'P':2,'S':3}

def map_values(x):
    return Mapping.get(x, x)

In [5]:
vectorized_map_values = np.vectorize(map_values)
converted_data = vectorized_map_values(data)

In [6]:
converted_data.dtype

dtype('float64')

In [7]:
segments = []

for i in range(0, converted_data.shape[1] - 20):
    segment = converted_data[0][i:i+20]
    segments.append(segment)


In [8]:
segments

[array([0., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1.,
        2., 2., 3.]),
 array([1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2.,
        2., 3., 1.]),
 array([2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2.,
        3., 1., 1.]),
 array([2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3.,
        1., 1., 2.]),
 array([3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1.,
        1., 2., 2.]),
 array([1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1.,
        2., 2., 3.]),
 array([1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2.,
        2., 3., 1.]),
 array([2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2.,
        3., 1., 1.]),
 array([2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3.,
        1., 1., 2.]),
 array([3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1., 1., 2., 2., 3., 1.,
        1., 2., 2.]),
 array([1., 1., 2., 

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(converted_data)

In [10]:
BATCH_SIZE = 20
batched_data= dataset.batch(BATCH_SIZE+1, drop_remainder=True)

In [11]:
def split_input_target(chunk):  
    input_text = chunk[:-1]  
    target_text = chunk[1:]  
    return input_text, target_text

dataset = batched_data.map(split_input_target)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [12]:
dataset

<MapDataset shapes: ((20, 200000), (20, 200000)), types: (tf.float64, tf.float64)>

In [13]:
batched_data


<BatchDataset shapes: (21, 200000), types: tf.float64>

In [22]:
data = dataset.shuffle(1000).batch(20,drop_remainder = True)

In [20]:
data

<BatchDataset shapes: ((20, 20, 200000), (20, 20, 200000)), types: (tf.float64, tf.float64)>

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

VOCAB_SIZE = 4
EMBEDDING_DIM = 32
RNN_UNITS = 1024

In [15]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (20, None, 32)            128       
_________________________________________________________________
lstm (LSTM)                  (20, None, 1024)          4329472   
_________________________________________________________________
dense (Dense)                (20, None, 4)             4100      
Total params: 4,333,700
Trainable params: 4,333,700
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(data,epochs = 10) 
#next attempt check what datasets can go into this function, make sure I am puttin the right stuff

Epoch 1/10


UnboundLocalError: local variable 'logs' referenced before assignment

In [ ]:
model.input_shape